In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
all_data = pd.concat([train, test], keys=['train', 'test'])
all_data

In [9]:
train.isnull().sum()

In [29]:
signs = ['Sex', 'Pclass', 'SibSp', 'Parch']
x_train = pd.get_dummies(train[signs])
x_test = pd.get_dummies(test[signs])
y_train = train['Survived']
x_train

In [30]:
from sklearn.utils import all_estimators
all_estimators(type_filter="classifier")

In [31]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
kf = KFold(n_splits=5, shuffle=True, random_state=1)

In [32]:
writer = open('./all_estimators_classifier.txt', 'w', encoding="utf-8")
writer.write('name\taccuracy\n')

In [33]:
import sys
for (name,Estimator) in all_estimators(type_filter="classifier"):

    try:
        model = Estimator()
        if 'score' not in dir(model):
            continue
        scores = cross_validate(model, x_train, y_train, cv=kf, scoring=['accuracy'])
        accuracy = scores['test_accuracy'].mean()
        writer.write(name + "\t" + str(accuracy) + '\n')
    except:
        print(sys.exc_info())
        print(name)
        pass

writer.close()

In [34]:
seed = 0

In [39]:
from sklearn.svm import SVC, NuSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

# サポートベクターマシン（SVM）
svm_clf = SVC(probability=True, random_state=seed)
svm_clf.fit(x_train, y_train)

nu_clf = NuSVC(probability=True, random_state=seed)
nu_clf.fit(x_train, y_train)

gaussian_clf = GaussianNB()
gaussian_clf.fit(x_train, y_train)

mlpc_clf = MLPClassifier()
mlpc_clf.fit(x_train, y_train)

In [42]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
dtc = DecisionTreeClassifier(random_state=seed)
estimators=[('svm', svm_clf), ('nuSVC', nu_clf), ('gaussianNB', gaussian_clf), ('mlpc', mlpc_clf)]
vote_clf=VotingClassifier(estimators=estimators, voting='soft', weights=[3, 3, 2, 1])
vote_clf.fit(x_train, y_train)
x_pred = vote_clf.predict(x_test)  
output = pd.DataFrame({'PassengerId':test.PassengerId, 'Survived':x_pred})
output.to_csv('submission.csv', index=False)